In [14]:
# @title Parameters
LOCATION = "us-central1" #@param {type:"string"}

PROJECT_ID = !gcloud projects list --filter="$(gcloud config get-value project)" --format="value(PROJECT_ID)"

if len(PROJECT_ID) > 1:
    PROJECT_ID = PROJECT_ID[1]
else:
    PROJECT_ID = PROJECT_ID[0]
print(PROJECT_ID)

PROJECT_NUMBER = !gcloud projects list --filter="$(gcloud config get-value project)" --format="value(PROJECT_NUMBER)"
if len(PROJECT_NUMBER) > 1:
    PROJECT_NUMBER = PROJECT_NUMBER[1]
else:
    PROJECT_NUMBER = PROJECT_NUMBER[0]

print(PROJECT_NUMBER)

TOKEN_ENDPOINT_URI = 'https://www.googleapis.com/oauth2/v3/tokeninfo?access_token='

IMAGEGEN_MODEL = "imagegeneration@002" #models available: imagegeneration@001 and imagegeneration@002
ENDPOINT_URL = f'projects/{PROJECT_NUMBER}/locations/{LOCATION}/publishers/google/models/{IMAGEGEN_MODEL}'

IMAGE_TEXT_MODEL = "imagetext"
IMAGEN_ENDPOINT_URL = f'projects/{PROJECT_NUMBER}/locations/{LOCATION}/publishers/google/models/{IMAGE_TEXT_MODEL}'


RATE_LIMIT = 10 #default quota 10 Online prediction requests per base model per minute per region per base_model

ml-demo-384110
1008225662928


In [2]:
import base64
import json
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib import rcParams
import numpy as np
import requests
from io import BytesIO
from IPython.display import Image
from PIL import Image
from IPython import display
from IPython.display import Image, display

from google.cloud import storage

In [15]:
import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION)

"""
Get access token
"""
def get_accessToken():
    global ACCESS_TOKEN
    gcloud_token = !gcloud auth print-access-token
    gcloud_tokeninfo = requests.get(TOKEN_ENDPOINT_URI + gcloud_token[0]).json()
    print("Access Token is generated for ", gcloud_tokeninfo['email'])
    ACCESS_TOKEN = gcloud_token[0]
    return ACCESS_TOKEN

ACCESS_TOKEN = get_accessToken()
#print(headers)

Access Token is generated for  admin@julienmiquel.altostrat.com


In [4]:

"""
encoding = base64.b64encode(open(img_path, 'rb').read())
The following function takes an image file path and returns a base64 encoded image as string
"""
def encode_image(img_path):
    with open(img_path, "rb") as img_file:
        encoded_str = base64.b64encode(img_file.read())
    return encoded_str

"""
decoding = base64.b64decode(encoding)
The following function takes a base64 encoded image and returns a PIL image
"""
def decode_image(img_base64):
    img_bytes = base64.b64decode(img_base64)
    return Image.open(BytesIO(img_bytes))

"""
display_image = decode_image(encoding)
The following function takes a PIL image and displays it
"""
def display_image(img):
    display.display(img)
    display.clear_output(wait=True)

"""
Read files from GCS bucket for a given path.
"""
def read_file_from_gcs(bucket_name, file_path):
    storage_client = storage.Client()
    bucket = storage_client.get_bucket(bucket_name)
    blob = bucket.blob(file_path)
    return blob

"""
Read b64 encoded file from GCS bucket
"""
def read_b64_from_gcs(bucket_name, file_path):
    blob = read_file_from_gcs(bucket_name, file_path)    
    return base64.b64encode(blob.download_as_bytes())

"""
Read Json Config file from GCS bucket
"""
def read_json_from_gcs(bucket_name, file_path):
    blob = read_file_from_gcs(bucket_name, file_path)
    str_json = blob.download_as_string()
    print(str_json)
    json_config = json.loads(str_json)
    return json_config


# Utility functions for Embeddings API with rate limiting
def rate_limit(max_per_minute):
    import time

    period = 60 / max_per_minute
    print("Waiting")
    while True:
        before = time.time()
        yield
        after = time.time()
        elapsed = after - before
        sleep_time = max(0, period - elapsed)
        if sleep_time > 0:
            print(".", end="")
            time.sleep(sleep_time)

In [16]:
# @title Setup

import json
import requests
import time

from IPython import display
import base64

# @title image Generation Model
def generate_image(
    prompt: str,
    sampleImageSize: str,
    sampleCount: int,
    seed: int
    ):
    rate_limit(RATE_LIMIT) 
    headers = {
      'Authorization': f'Bearer {ACCESS_TOKEN}',
      'Content-Type': 'application/json; charset=UTF-8'
    }
    data = {"instances": [{"prompt": prompt}],
            "parameters": {"sampleImageSize": sampleImageSize,"sampleCount": sampleCount, "seed": seed}}

    response = requests.post(f'https://{LOCATION}-aiplatform.googleapis.com/v1/{ENDPOINT_URL}:predict', data=json.dumps(data), headers=headers)
    return response

###
# Visual Question Answering (VQA) to get image information
##
def visual_qa(
    prompt: str,
    sampleCount: int,
    encoded_str: str
    ):
    rate_limit(RATE_LIMIT) 
    
    headers = {
      'Authorization': f'Bearer {ACCESS_TOKEN}',
      'Content-Type': 'application/json; charset=UTF-8'
    }
    
    data = {
        "instances": [
            {
            "prompt": prompt,
            "image": {
                "bytesBase64Encoded": encoded_str
            }
            }
        ],
        "parameters": {
            "sampleCount": sampleCount
            }
        }

    response = requests.post(f'https://{LOCATION}-aiplatform.googleapis.com/v1/{IMAGEN_ENDPOINT_URL}:predict', 
        data=json.dumps(data), headers=headers)

    return response

###
#Visual captioning is available in the following languages:
# English (en)
# French (fr)
# German (de)
# Italian (it)
# Spanish (es)
##
def visual_captioning(
    sampleCount: int,
    language: str, 
    # English (en), French (fr), German (de), Italian (it), Spanish (es)
    encoded_str: str
    ):
    rate_limit(RATE_LIMIT) 

    headers = {
      'Authorization': f'Bearer {ACCESS_TOKEN}',
      'Content-Type': 'application/json; charset=UTF-8'
    }
    
    data = {
        "instances": [
            {
            "image": {
                "bytesBase64Encoded": encoded_str
            }
            }
        ],
        "parameters": {
            "sampleCount": sampleCount,
            "language": language
            }
        }

    response = requests.post(f'https://{LOCATION}-aiplatform.googleapis.com/v1/{IMAGEN_ENDPOINT_URL}:predict', 
        data=json.dumps(data), headers=headers)

    return response

def display_images(
    predictions: str,
    sampleImageSize: str,
    scale: float,
    ):
    width = int(sampleImageSize) * scale
    height = int(sampleImageSize) * scale
    for index, key in enumerate(predictions):
        display.display(display.Image(base64.b64decode(predictions[index]['bytesBase64Encoded']), width = width, height = height))
        print()

def print_text_response(response):
    if response.status_code == 200:
        json_response = json.loads(response.text)
        #print(json_response)
        predictions = json_response['predictions']
        print(predictions)
        return predictions
    print(response)
    response.
    return ""


In [17]:

base_image_path = "/Users/julienmiquel/dev/github-genAI/genAI/examples/B5CL-GALLERY-0-2.jpg"

print("visual_captioning:")
print_text_response(visual_captioning( 3, "fr",str(encode_image(base_image_path), 'UTF-8')))

print("visual_qa:")
print_text_response(visual_qa("What is the color of the product ?", 3,str(encode_image(base_image_path), 'UTF-8')))



visual_captioning:
<Response [400]>
visual_qa:
<Response [400]>


''

In [ ]:
# @title Prediction for the Logo
prompt = f"""Design of abstract logo featuring a dog in blue on a pink background. 
Include lines as an additional design element. """
size = 256 

print(prompt)
response = generate_image(prompt,
                                    size, # sampleImageSize
                                    4, # sampleCount
                                    0  # seed
                                    )
print(response)
json_response = json.loads(response.text)
predictions = json_response['predictions']
display_images(predictions,size,1)